In [1]:
import PySimpleGUI as sg
import oct_tools as ot
from oct_tools import Mzi, Bpd, LightSource
from scipy.signal import savgol_filter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import figure
%matplotlib qt

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# plt.style.use('dark_background')
plt.style.use('bmh')
# Set the background color to dark
plt.rcParams['axes.facecolor'] = '#000000'
plt.rcParams['figure.facecolor'] = '#1a1a1a'
# # Set legend text color to white
# plt.rcParams['legend.facecolor'] = 'none'  # Set legend background color to transparent
# plt.rcParams['legend.labelcolor'] = 'white'  # Set legend label color to white


In [2]:
## VARIABLE SETUP
EXITALL = 'EXITALL'

INPUT_POWER_INPUT = 'INPUT_POWER_INPUT'
SWEEP_FREQUENCY_INPUT = 'SWEEP_FREQUENCY_INPUT'
SWEEP_PARAM_INPUT_SIZE = 5
SWEEP_PARAM_INPUT_FONT_SIZE = 20

LIGHT_SOURCE_RADIO_GROUP = 'LIGHT_SOURCE_RADIO_GROUP'
LIGHT_SOURCE_DEFINE_BUTTON = 'LIGHT_SOURCE_DEFINE_BUTTON'
LIGHT_SOURCE_DEFINE_RADIO = 'LIGHT_SOURCE_DEFINE_RADIO'
LIGHT_SOURCE_LOAD_INPUT = 'LIGHT_SOURCE_LOAD_INPUT'
LIGHT_SOURCE_LOAD_RADIO = 'LIGHT_SOURCE_LOAD_RADIO'
LIGHT_SOURCE_SWEEP_FREQUENCY_INPUT = 'LIGHT_SOURCE_SWEEP_FREQUENCY_INPUT'
LIGHT_SOURCE_DUTY_CYCLE_INPUT = 'LIGHT_SOURCE_DUTY_CYCLE_INPUT'
LIGHT_SOURCE_WAVELENGTH_START_INPUT = 'LIGHT_SOURCE_WAVELENGTH_START_INPUT'
LIGHT_SOURCE_WAVELENGTH_STOP_INPUT = 'LIGHT_SOURCE_WAVELENGTH_STOP_INPUT'
LIGHT_SOURCE_OUTPUT_POWER_INPUT = 'LIGHT_SOURCE_OUTPUT_POWER_INPUT'
LIGHT_SOURCE_REFLECTION_DEPTH_INPUT = 'LIGHT_SOURCE_REFLECTION_DEPTH_INPUT'
LIGHT_SOURCE_REFLECTION_STRENGTH_INPUT = 'LIGHT_SOURCE_REFLECTION_STRENGTH_INPUT'
LIGHT_SOURCE_WAVELENGTH_NOISE_INPUT = 'LIGHT_SOURCE_WAVELENGTH_NOISE_INPUT'
LIGHT_SOURCE_RUN_BUTTON = 'LIGHT_SOURCE_RUN_BUTTON'
LIGHT_SOURCE_CLOSE_BUTTON = 'LIGHT_SOURCE_CLOSE_BUTTON'



MZI_LENGTH_SLIDER = 'MZI_LENGTH_SLIDER'
MZI_LENGTH_SLIDER_DEFAULT = 4.0 # mm
MZI_LENGTH_SLIDER_RESOLUTION = 0.1
MZI_LENGTH_SLIDER_RANGE = (0.1, 100) # mm
MZI_LENGTH_SLIDER_INPUT = 'MZI_LENGTH_SLIDER_INPUT'
MZI_LENGTH_INPUT_SIZE = 6
MZI_FSR_TEXT = 'MZI_FSR_TEXT'

MEAS_DISTANCE_SLIDER = 'MEAS_DISTANCE_SLIDER'
MEAS_DISTANCE_SLIDER_DEFAULT = 2.0 # mm
MEAS_DISTANCE_SLIDER_RESOLUTION = MZI_LENGTH_SLIDER_RESOLUTION / 2
MEAS_DISTANCE_SLIDER_RANGE = (0.1, MZI_LENGTH_SLIDER_RANGE[1] / 2) # mm
MEAS_DISTANCE_SLIDER_INPUT = 'MEAS_DISTANCE_SLIDER_INPUT'
MEAS_DISTANCE_INPUT_SIZE = 6
MEAS_FSR_TEXT = 'MEAS_FSR_TEXT'

INPUT_COUPLER_H = 'INPUT_COUPLER_H'
INPUT_COUPLER_L = 'INPUT_COUPLER_L'
OUTPUT_COUPLER_H = 'OUTPUT_COUPLER_H'
OUTPUT_COUPLER_L = 'OUTPUT_COUPLER_L'

CROP_THRESHOLD_INPUT = 'CROP_THRESHOLD_INPUT'


BPD_BANDWIDTH_INPUT = 'BPD_BANDWIDTH_INPUT'
BPD_INPUT_SIZE = 7

BPD_GAIN_SLIDER = 'BPD_GAIN_SLIDER'
BPD_GAIN_SLIDER_DEFAULT = '5000'
BPD_GAIN_SLIDER_RESOLUTION = 100
BPD_GAIN_SLIDER_RANGE = (100, 20_000)
BPD_GAIN_SLIDER_INPUT = 'BPD_GAIN_SLIDER_INPUT'
BPD_GAIN_INPUT_SIZE = 7

BPD_NOISE_SLIDER = 'BPD_NOISE_SLIDER'
BPD_NOISE_SLIDER_DEFAULT = 5e-12
BPD_NOISE_SLIDER_RESOLUTION = 1e-12
BPD_NOISE_SLIDER_RANGE = (0, 30e-12)
BPD_NOISE_SLIDER_INPUT = 'BPD_NOISE_SLIDER_INPUT'
BPD_NOISE_INPUT_SIZE = 7

RUN_BUTTON = 'RUN_BUTTON'
RUN_BUTTON_SIZE = (25, 4)
RUN_BUTTON_FONT_SIZE = '72'

FRINGE_CANVAS = 'FRINGE_CANVAS'
FFT_CANVAS = 'FFT_CANVAS'
FRINGE_PLOT_POPOUT_BUTTON = 'FRINGE_PLOT_POPOUT_BUTTON'
FFT_PLOT_POPOUT_BUTTON = 'FFT_PLOT_POPOUT_BUTTON'
PLOT_STYLE = 'PLOT_STYLE'

BUTTON_SIZE = (10, 2)
SELECTED_BUTTON_COLOR = sg.PURPLES[0]

BUTTON_SIZE = (10, 2)


sg.theme('Topanga');

In [3]:
# ------------------------------- WINDOWS -------------------------------
def make_main_window() -> sg.Window:
    ''' Makes main window
    '''
    sweep_parameter_text_column_layout = [[sg.Text('Input Power (mW)')],
                                          [sg.Text('Sweep Frequency (kHz)')],
                                          ]
    sweep_parameter_input_column_layout = [[sg.Input('2.5', key=INPUT_POWER_INPUT, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE, disabled=True)],
                                           [sg.Input('50', key=SWEEP_FREQUENCY_INPUT, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE, disabled=True)],
                                           ]
    sweep_parameters_frame_layout = [[sg.Radio('', LIGHT_SOURCE_RADIO_GROUP, key=LIGHT_SOURCE_DEFINE_RADIO, default=True, enable_events=True), sg.Button('Define Light Source', key=LIGHT_SOURCE_DEFINE_BUTTON)],
                                     [sg.Push(), sg.Text('OR'), sg.Push()],
                                     [sg.Radio('', LIGHT_SOURCE_RADIO_GROUP, key=LIGHT_SOURCE_LOAD_RADIO, enable_events=True),
                                      sg.Input('', key=LIGHT_SOURCE_LOAD_INPUT, visible=False, enable_events=True), sg.FileBrowse('Load Light Source', file_types=(('Lin Data csv', '*LinData.csv'),),)],
                                      [sg.Column(sweep_parameter_text_column_layout), sg.Column(sweep_parameter_input_column_layout)],                                     
                                    ]

    
    mzi_text_column_layout = [[sg.Text('Input Coupler Split (%)')],
                              [sg.Text('Output Coupler Split (%)')],
                              ]
    mzi_input_column_layout = [[sg.Input('50', key=INPUT_COUPLER_H, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE), sg.Input('50', key=INPUT_COUPLER_L, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                               [sg.Input('50', key=OUTPUT_COUPLER_H, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE), sg.Input('50', key=OUTPUT_COUPLER_L, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                               ]
    mzi_frame_layout = [[sg.Text('Length in Air (mm)')],
                        [sg.Slider(MZI_LENGTH_SLIDER_RANGE, key=MZI_LENGTH_SLIDER, default_value=MZI_LENGTH_SLIDER_DEFAULT, enable_events=True, orientation='h', resolution=MZI_LENGTH_SLIDER_RESOLUTION, disable_number_display=True),
                         sg.Input(MZI_LENGTH_SLIDER_DEFAULT, key=MZI_LENGTH_SLIDER_INPUT, size=MZI_LENGTH_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE, enable_events=True)],
                         [sg.Text('FSR:'), sg.Text('', key=MZI_FSR_TEXT), sg.Text('pm')],
                        [sg.Column(mzi_text_column_layout), sg.Column(mzi_input_column_layout)],
                        [sg.Text('Measurement Distance (mm)')],
                        [sg.Slider(MEAS_DISTANCE_SLIDER_RANGE, key=MEAS_DISTANCE_SLIDER, default_value=MEAS_DISTANCE_SLIDER_DEFAULT, enable_events=True, orientation='h', resolution=MEAS_DISTANCE_SLIDER_RESOLUTION, disable_number_display=True),
                         sg.Input(MEAS_DISTANCE_SLIDER_DEFAULT, key=MEAS_DISTANCE_SLIDER_INPUT, size=MEAS_DISTANCE_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE, enable_events=True)],
                        [sg.Text('FSR:'), sg.Text('', key=MEAS_FSR_TEXT), sg.Text('pm')],
                        ]

    bpd_text_column_layout = [[sg.Text('Bandwidth (Hz)')],
                            ]
    bpd_input_column_layout = [[sg.Input('500e6', key=BPD_BANDWIDTH_INPUT, size=BPD_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                            ]
    bpd_frame_layout = [[sg.Column(bpd_text_column_layout), sg.Column(bpd_input_column_layout)],
                        [sg.Text('Gain (V/A)')],
                        [sg.Slider(BPD_GAIN_SLIDER_RANGE, key=BPD_GAIN_SLIDER, default_value=BPD_GAIN_SLIDER_DEFAULT, enable_events=True, orientation='h', resolution=BPD_GAIN_SLIDER_RESOLUTION, disable_number_display=True),
                         sg.Input(BPD_GAIN_SLIDER_DEFAULT, key=BPD_GAIN_SLIDER_INPUT, size=BPD_GAIN_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE, enable_events=True)],
                        [sg.Text('Noise Density (A/rtHz)')],
                        [sg.Slider(BPD_NOISE_SLIDER_RANGE, key=BPD_NOISE_SLIDER, default_value=BPD_NOISE_SLIDER_DEFAULT, enable_events=True, orientation='h', resolution=BPD_NOISE_SLIDER_RESOLUTION, disable_number_display=True),
                         sg.Input(BPD_NOISE_SLIDER_DEFAULT, key=BPD_NOISE_SLIDER_INPUT, size=BPD_NOISE_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE, enable_events=True)],
                        ]

    
    misc_frame_layout = [[sg.Text('Crop Threshold (%)'), sg.Input('5', key=CROP_THRESHOLD_INPUT, size=MEAS_DISTANCE_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)]]

    
    entry_column_layout = [[sg.Frame('Sweep Parameters', sweep_parameters_frame_layout)], 
                        [sg.Frame('Reference MZI Parameters', mzi_frame_layout)],
                        [sg.Frame('Misc.', misc_frame_layout)],
                        [sg.Frame('Balanced Detector', bpd_frame_layout)],
                        [sg.Text()],
                        [sg.Push(), sg.Button('Run', key=RUN_BUTTON, size=RUN_BUTTON_SIZE, font=RUN_BUTTON_FONT_SIZE), sg.Push()]]


    frame_layout_plotting_area = [[sg.Canvas(key=FRINGE_CANVAS, background_color='black'), sg.Button('Popout', key=FRINGE_PLOT_POPOUT_BUTTON)],
                                  [sg.Canvas(key=FFT_CANVAS, size=(640, 480), background_color='black'), sg.Button('Popout', key=FFT_PLOT_POPOUT_BUTTON)],
                                  [sg.Push(), sg.Combo(plt.style.available, size=(15, 10), key=PLOT_STYLE, enable_events=True)]]


    layout = [[sg.Push(), sg.Text('OCT Sandbox', font=48), sg.Push()],
              [sg.Column(entry_column_layout, vertical_alignment='top'), sg.Push(), sg.Frame('Plotting', frame_layout_plotting_area)],
              [sg.VPush()],
              [sg.Button('Exit', key=EXITALL, size=BUTTON_SIZE, font=24)]]

    return sg.Window('OCT Sandbox', layout, resizable=True, finalize=True)

def make_light_source_window() -> sg.Window:
    ''' Makes light source definition window
    '''
    sweep_parameter_text_column_layout = [[sg.Text('Sweep Frequency (Hz)')], [sg.VPush()],
                                          [sg.Text('Duty Cycle')], [sg.VPush()],
                                          [sg.Text('Wavelength Start Stop(nm)')], [sg.VPush()],
                                          [sg.Text('Output Power (mW)')], [sg.VPush()],
                                          [sg.Text('Reflection Depth')], [sg.VPush()],
                                          [sg.Text('Reflection Strength (dBc)')], [sg.VPush()],
                                          [sg.Text('Wavelength Noise Level (pm/Hz)')],
                                          ]
    sweep_parameter_input_column_layout = [[sg.Input('100e3', key=LIGHT_SOURCE_SWEEP_FREQUENCY_INPUT, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                                           [sg.Input('0.65', key=LIGHT_SOURCE_DUTY_CYCLE_INPUT, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                                           [sg.Input('1000', key=LIGHT_SOURCE_WAVELENGTH_START_INPUT, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE),
                                            sg.Input('1100', key=LIGHT_SOURCE_WAVELENGTH_STOP_INPUT, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                                           [sg.Input('30', key=LIGHT_SOURCE_OUTPUT_POWER_INPUT, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                                           [sg.Input('1e-3', key=LIGHT_SOURCE_REFLECTION_DEPTH_INPUT, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                                           [sg.Input('0', key=LIGHT_SOURCE_REFLECTION_STRENGTH_INPUT, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                                           [sg.Input('0', key=LIGHT_SOURCE_WAVELENGTH_NOISE_INPUT, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                                           ]
    sweep_parameters_frame_layout = [[sg.Column(sweep_parameter_text_column_layout), sg.Column(sweep_parameter_input_column_layout)],
                                    ]
    
        
    entry_column_layout = [[sg.Frame('Sweep Parameters', sweep_parameters_frame_layout)],]


    layout = [[sg.Push(), sg.Text('Light Source Definition', font=48), sg.Push()],
              [sg.Column(entry_column_layout, vertical_alignment='top')],
              [sg.VPush()],
              [sg.Button('Close', key=LIGHT_SOURCE_CLOSE_BUTTON, size=BUTTON_SIZE, font=24), sg.Button('Apply & Run', key=LIGHT_SOURCE_RUN_BUTTON, size=BUTTON_SIZE, font=24)]]
    

    return sg.Window('Light Source Definition', layout, finalize=True)

def close_all_windows(windows : list[sg.Window]) -> None:
    for window in windows:
        if window is not None:
            window.close()

# ------------------------------- PSG HELPERS -------------------------------
def draw_figure(canvas, figure) -> FigureCanvasTkAgg:
    figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
    figure_canvas_agg.draw()
    figure_canvas_agg.get_tk_widget().pack()#side='top', fill='both', expand=1)
    return figure_canvas_agg


# ------------------------------- PLOTTING -------------------------------
def make_figures(window: sg.Window):
    fig0 = figure.Figure(figsize=(13, 5), dpi=100)
    ax0 = fig0.add_subplot(111)
    fig1 = figure.Figure(figsize=(13, 5), dpi=100)
    ax1 = fig1.add_subplot(111)

    fringe_fig_canvas_agg = draw_figure(window[FRINGE_CANVAS].TKCanvas, fig0)
    fft_fig_canvas_agg = draw_figure(window[FFT_CANVAS].TKCanvas, fig1)

    return fig0, ax0, fig1, ax1, fringe_fig_canvas_agg, fft_fig_canvas_agg

def plot_fringe(t, reference_voltage, meas_voltage, fig_canvas_agg, ax, popout=False) -> None:
    if popout:
        # create new fig with pyplot
        fig = plt.figure()
        ax = plt.axes()
        plt.plot(t, reference_voltage)
        plt.plot(t, meas_voltage)
    ax.cla()
    ax.plot(t, reference_voltage, label='Reference Fringe')
    ax.plot(t, meas_voltage, label='Measurement Fringe', alpha=0.5)
    ax.set_ylabel('Detection Voltage (V)')
    ax.set_xlabel('Time (s)')
    ax.legend()
    if popout:
        plt.show()
    else:
        fig_canvas_agg.draw()

def plot_fft(freq, reference_fft, meas_fft, fig_canvas_agg, ax, popout=False) -> None:
    if popout:
        # create new fig with pyplot
        fig = plt.figure()
        ax = plt.axes()
        plt.plot(freq, reference_fft)
        plt.plot(freq, meas_fft)
    ax.cla()
    ax.plot(freq, reference_fft, label='Reference', linewidth=0.5)
    ax.plot(freq, meas_fft, label='Measurement', linewidth=0.5)
    ax.set_ylabel('Magnitude')
    ax.set_xlabel('Frequency (Hz)')
    ax.legend()
    if popout:
        plt.show()
    else:
        fig_canvas_agg.draw()

# ------------------------------- DATA PROCESSING -------------------------------
def calculate_fft(T, signal):
    '''T is sample spacing of signal
    Returns frequency, fft complex data
    '''
    n = len(signal)
    freq = np.fft.fftfreq(n, T)[: n // 2]  # Frequency values
    fft_result = np.fft.fft(signal)[: n // 2] / n  # FFT result (normalized)

    return freq, fft_result

def get_lin_data_arrays(lin_data, average_power_in):
    t_data = lin_data['Time'].to_numpy()
    wl = lin_data['Wavelength'].to_numpy() * 1e-9
    boa_out = np.abs(lin_data['PowerEnvelope'].to_numpy())
    boa_out = boa_out / np.max(boa_out)
    boa_out = boa_out * average_power_in / np.mean(boa_out) # normalize to average power

    # increase resolution to avoid wavelength aliasing through long MZI
    npts = int(1e6)
    # interpolate
    t_new = ot.resize_interpolate(t_data, npts)
    wl = ot.resize_interpolate(wl, npts)
    boa_out = ot.resize_interpolate(boa_out, npts)

    
    return t_new, wl, boa_out


In [4]:
main_window, light_source_window, = make_main_window(), None,

fig0, ax0, fig1, ax1, fringe_fig_canvas_agg, fft_fig_canvas_agg = make_figures(main_window)


_, values = main_window.read(timeout=0)
lin_data = pd.read_csv(r'B4363_V4367_Final_LinData.csv')
ref_mzi = Mzi(float(values[MZI_LENGTH_SLIDER_INPUT]) * 1e-3, float(values[INPUT_COUPLER_H]) / 100, float(values[OUTPUT_COUPLER_H]) / 100)
meas_mzi = Mzi(float(values[MEAS_DISTANCE_SLIDER_INPUT]) * 1e-3, float(values[INPUT_COUPLER_H]) / 100, float(values[OUTPUT_COUPLER_H]) / 100)
bpd = Bpd(i_noise = float(values[BPD_NOISE_SLIDER_INPUT]))
light_source = LightSource(p_ave=30e-3, wavelength_start=1260e-9, wavelength_stop=1360e-9, sweep_rate=100e3, duty_cycle=0.65, reflection_depth=1e-3, reflection_strength=0)

main_window.write_event_value(RUN_BUTTON, '')
while True:
    window, event, values = sg.read_all_windows()
    print(event)
    
    if window == main_window and event in (sg.WIN_CLOSED, EXITALL):
        break
    
    if window == main_window:
        if event == RUN_BUTTON:
            # TODO: bundle this up neatly
            # Update
            ref_mzi = Mzi(float(values[MZI_LENGTH_SLIDER_INPUT]) * 1e-3, float(values[INPUT_COUPLER_H]) / 100, float(values[OUTPUT_COUPLER_H]) / 100)
            meas_mzi = Mzi(float(values[MEAS_DISTANCE_SLIDER_INPUT]) * 2 * 1e-3, float(values[INPUT_COUPLER_H]) / 100, float(values[OUTPUT_COUPLER_H]) / 100)
            bpd = Bpd(gain = float(values[BPD_GAIN_SLIDER_INPUT]), bandwidth = float(values[BPD_BANDWIDTH_INPUT]), i_noise = float(values[BPD_NOISE_SLIDER_INPUT]))

            # Calculate all
            if values[LIGHT_SOURCE_LOAD_RADIO]:
                t, wl, power_out = get_lin_data_arrays(lin_data, float(values[INPUT_POWER_INPUT]))
            elif values[LIGHT_SOURCE_DEFINE_RADIO]:
                t, wl, power_out = light_source.generate_vectors()
            # power_out = savgol_filter(power_out, 501, 1) # TODO: handle filtering for different inputs
            t, ref_pout1, ref_pout2 = ref_mzi.calculate_from_time(t, wl, power_out)
            t, meas_pout1, meas_pout2 = meas_mzi.calculate_from_time(t, wl, power_out) # TODO: Need to interpolate to have same length array for all MZIs
            
            v_ref = bpd.detect(ref_pout1, ref_pout2) # TODO: don't need this if below in averaging
            v_meas = bpd.detect(meas_pout1, meas_pout2) # TODO: don't need this if below in averaging

            # Window
            idx = ot.get_crop_indices(power_out, min(float(values[CROP_THRESHOLD_INPUT]) / 100, 0.5))
            # Calculate FFTs
            ref_signal = ot.apply_hanning(v_ref[idx[0] : idx[1]])

            # TODO: fft averages (efficiently)
            n_avgs = 1
            fft_freq = calculate_fft(t[1] - t[0], ref_signal)[0]
            
            ref_fft = []
            meas_fft = []
            for i in range(n_avgs):
                v_ref = bpd.detect(ref_pout1, ref_pout2)
                v_meas = bpd.detect(meas_pout1, meas_pout2)
                # Window
                idx = ot.get_crop_indices(power_out, min(float(values[CROP_THRESHOLD_INPUT]) / 100, 0.5))
                # Calculate FFTs
                ref_signal = ot.apply_hanning(v_ref[idx[0] : idx[1]])
                meas_signal = ot.apply_hanning(v_meas[idx[0] : idx[1]])
                ref_fft.append(np.abs(calculate_fft(t[1], ref_signal)[1]))
                meas_fft.append(np.abs(calculate_fft(t[1], ot.resample(meas_signal, ref_signal))[1]))

            ref_fft = np.mean(ref_fft, axis=0)
            meas_fft = np.mean(meas_fft, axis=0)

            # Update display
            window[MZI_FSR_TEXT].update(str(round(ot.calculate_fsr_wavelength(np.mean(wl), ref_mzi.fsr) * 1e12, 1)))
            window[MEAS_FSR_TEXT].update(str(round(ot.calculate_fsr_wavelength(np.mean(wl), meas_mzi.fsr) * 1e12, 1)))
            # Plot
            plot_fringe(t, v_ref, v_meas, fringe_fig_canvas_agg, ax0)
            plot_fft(fft_freq, 10 * np.log10((np.abs(ref_fft))), 10 * np.log10(np.abs(meas_fft)), fft_fig_canvas_agg, ax1)

        elif event == FRINGE_PLOT_POPOUT_BUTTON:
            plot_fringe(t, v_ref, v_meas, fringe_fig_canvas_agg, ax0, popout=True)
        elif event == FFT_PLOT_POPOUT_BUTTON:
            plot_fft(fft_freq, 10 * np.log10((np.abs(ref_fft))), 10 * np.log10(np.abs(meas_fft)), fft_fig_canvas_agg, ax1, popout=True)
            
        # Light Source
        elif event == LIGHT_SOURCE_DEFINE_BUTTON:
            # check if window already open
            if light_source_window is None:
                # make light source definition window
                light_source_window = make_light_source_window()
            # activate define light source radio
            window[LIGHT_SOURCE_DEFINE_RADIO].update(True)
            
        elif event == LIGHT_SOURCE_LOAD_INPUT:
            # load lin_data
            lin_data = pd.read_csv(values[LIGHT_SOURCE_LOAD_INPUT])
            # activate load light source radio
            window[LIGHT_SOURCE_LOAD_RADIO].update(True)

        elif event == LIGHT_SOURCE_LOAD_RADIO:
            window[INPUT_POWER_INPUT].update(disabled=False)
            window[SWEEP_FREQUENCY_INPUT].update(disabled=False)
        elif event == LIGHT_SOURCE_DEFINE_RADIO:
            window[INPUT_POWER_INPUT].update(disabled=True)
            window[SWEEP_FREQUENCY_INPUT].update(disabled=True)

        # MZI Parameters
        elif event in (MZI_LENGTH_SLIDER, MEAS_DISTANCE_SLIDER, BPD_GAIN_SLIDER, BPD_NOISE_SLIDER): # update inputs to share the slider value
            main_window[event + '_INPUT'].update(values[event])
            window.write_event_value(RUN_BUTTON, '')
        elif event in (MZI_LENGTH_SLIDER_INPUT, MEAS_DISTANCE_SLIDER_INPUT, BPD_GAIN_SLIDER_INPUT, BPD_NOISE_SLIDER_INPUT):
            if values[event] == '':
                continue
            main_window[event.strip('_INPUT')].update(values[event])
            window.write_event_value(RUN_BUTTON, '')

        elif event == PLOT_STYLE:
            plt.style.use(values[PLOT_STYLE])
            plt.close('all')
            # plt.close(fig1)
            window.write_event_value(RUN_BUTTON, '')

    if window == light_source_window:
        if event in (LIGHT_SOURCE_CLOSE_BUTTON, sg.WIN_CLOSED):
            # close light_source_window and set to None
            light_source_window.close()
            light_source_window = None

        elif event == LIGHT_SOURCE_RUN_BUTTON:
            main_window.write_event_value(RUN_BUTTON, '')

windows = (main_window, light_source_window,)
close_all_windows(windows)

RUN_BUTTON
RUN_BUTTON
None
